# Model confidence set (MCS) of heterogeneous models
In this notebook we compute the Model confidence set of each cryptocurrencies' models

In [21]:
# import libraries
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
from arch.bootstrap import MCS as MCS2

In [22]:
import numpy as np
from numpy.random import rand
from numpy import ix_
import pandas as pd

In [23]:
# Global variables
btc_path = './models/ann/v3/BTC/'
eos_path = './models/ann/v3/EOS/'
eth_path = './models/ann/v3/ETH/'
iota_path = './models/ann/v3/IOTA/'

In [24]:
crypto_list = [btc_path,eth_path,iota_path,eos_path]

In [25]:
def read_csv_files_and_create_one_df(csv_list, path):
    """Get the list of files in the folder."""
    main_df = pd.DataFrame()
    for filename in csv_list:
        sliced_filename = filename.split("-")
        ann_model_number = sliced_filename[0]
        ann_drate = sliced_filename[2]        
        ann_opt = sliced_filename[4]        
        ts_model = sliced_filename[5]        
        ts_p = sliced_filename[6]        
        ts_q = sliced_filename[7]  
        ts_dist = sliced_filename[8].split('.')[0]
        df_curr = pd.read_csv(f"{path}csv/{filename}", index_col=0)
        main_df[f"{ann_model_number}-drate-{ann_drate}-opt-{ann_opt}-{ts_model}-{ts_p}-{ts_q}-{ts_dist}"] = (df_curr['ann'] - df_curr['hvol14'])**2
        main_df[f"{ts_model}-{ts_p}-{ts_q}-{ts_dist}"] = (df_curr['ts'] - df_curr['hvol14'])**2
        
    return main_df

In [26]:
def bootstrap_sample(data, B, w):
    '''
    Bootstrap the input data
    data: input numpy data array
    B: boostrap size
    w: block length of the boostrap
    '''
    t = len(data)
    p = 1 / w
    indices = np.zeros((t, B), dtype=int)
    indices[0, :] = np.ceil(t * rand(1, B))
    select = np.asfortranarray(rand(B, t).T < p)
    vals = np.ceil(rand(1, np.sum(np.sum(select))) * t).astype(int)
    indices_flat = indices.ravel(order="F")
    indices_flat[select.ravel(order="F")] = vals.ravel()
    indices = indices_flat.reshape([B, t]).T
    for i in range(1, t):
        indices[i, ~select[i, :]] = indices[i - 1, ~select[i, :]] + 1
    indices[indices > t] = indices[indices > t] - t
    indices -= 1
    return data[indices]


def compute_dij(losses, bsdata):
    '''Compute the loss difference'''
    t, M0 = losses.shape
    B = bsdata.shape[1]
    dijbar = np.zeros((M0, M0))
    for j in range(M0):
        dijbar[j, :] = np.mean(losses - losses[:, [j]], axis=0)

    dijbarstar = np.zeros((B, M0, M0))
    for b in range(B):
        meanworkdata = np.mean(losses[bsdata[:, b], :], axis=0)
        for j in range(M0):
            dijbarstar[b, j, :] = meanworkdata - meanworkdata[j]

    vardijbar = np.mean((dijbarstar - np.expand_dims(dijbar, 0)) ** 2, axis=0)
    vardijbar += np.eye(M0)

    return dijbar, dijbarstar, vardijbar


def calculate_PvalR(z, included, zdata0):
    '''Calculate the p-value of relative algorithm'''
    empdistTR = np.max(np.max(np.abs(z), 2), 1)
    zdata = zdata0[ix_(included - 1, included - 1)]
    TR = np.max(zdata)
    pval = np.mean(empdistTR > TR)
    return pval


def calculate_PvalSQ(z, included, zdata0):
    '''Calculate the p-value of sequential algorithm'''
    empdistTSQ = np.sum(z ** 2, axis=1).sum(axis=1) / 2
    zdata = zdata0[ix_(included - 1, included - 1)]
    TSQ = np.sum(zdata ** 2) / 2
    pval = np.mean(empdistTSQ > TSQ)
    return pval


def iterate(dijbar, dijbarstar, vardijbar, alpha, algorithm="R"):
    '''Iteratively excluding inferior model'''
    B, M0, _ = dijbarstar.shape
    z0 = (dijbarstar - np.expand_dims(dijbar, 0)) / np.sqrt(
        np.expand_dims(vardijbar, 0)
    )
    zdata0 = dijbar / np.sqrt(vardijbar)

    excludedR = np.zeros([M0, 1], dtype=int)
    pvalsR = np.ones([M0, 1])

    for i in range(M0 - 1):
        included = np.setdiff1d(np.arange(1, M0 + 1), excludedR)
        m = len(included)
        z = z0[ix_(range(B), included - 1, included - 1)]

        if algorithm == "R":
            pvalsR[i] = calculate_PvalR(z, included, zdata0)
        elif algorithm == "SQ":
            pvalsR[i] = calculate_PvalSQ(z, included, zdata0)

        scale = m / (m - 1)
        dibar = np.mean(dijbar[ix_(included - 1, included - 1)], 0) * scale
        dibstar = np.mean(dijbarstar[ix_(range(B), included - 1, included - 1)], 1) * (
            m / (m - 1)
        )
        vardi = np.mean((dibstar - dibar) ** 2, axis=0)
        t = dibar / np.sqrt(vardi)
        modeltoremove = np.argmax(t)
        excludedR[i] = included[modeltoremove]

    maxpval = pvalsR[0]
    for i in range(1, M0):
        if pvalsR[i] < maxpval:
            pvalsR[i] = maxpval
        else:
            maxpval = pvalsR[i]

    excludedR[-1] = np.setdiff1d(np.arange(1, M0 + 1), excludedR)
    pl = np.argmax(pvalsR > alpha)
    includedR = excludedR[pl:]
    excludedR = excludedR[:pl]
    return includedR - 1, excludedR - 1, pvalsR


def MCS(losses, alpha, B, w, algorithm):
    '''Main function of the MCS'''
    t, M0 = losses.shape
    bsdata = bootstrap_sample(np.arange(t), B, w)
    dijbar, dijbarstar, vardijbar = compute_dij(losses, bsdata)
    includedR, excludedR, pvalsR = iterate(
        dijbar, dijbarstar, vardijbar, alpha, algorithm=algorithm
    )
    return includedR, excludedR, pvalsR

In [1]:
class ModelConfidenceSet(object):
    def __init__(self, data, alpha, B, w, algorithm="SQ", names=None):
        """
        Implementation of Econometrica Paper:
        Hansen, Peter R., Asger Lunde, and James M. Nason. "The model confidence set." Econometrica 79.2 (2011): 453-497.
        Retrieved from : https://michael-gong.com/blogs/model-confidence-set/
        Input:
            data->pandas.DataFrame or numpy.ndarray: input data, columns are the losses of each model 
            alpha->float: confidence level
            B->int: bootstrap size for computation covariance
            w->int: block size for bootstrap sampling
            algorithm->str: SQ or R, SQ is the first t-statistics in Hansen (2011) p.465, and R is the second t-statistics
            names->list: the name of each model (corresponding to each columns). 

        Method:
            run(self): compute the MCS procedure

        Attributes:
            included: models that are in the model confidence sets at confidence level of alpha
            excluded: models that are NOT in the model confidence sets at confidence level of alpha
            pvalues: the bootstrap p-values of each models
        """

        if isinstance(data, pd.DataFrame):
            self.data = data.values
            self.names = data.columns.values if names is None else names
        elif isinstance(data, np.ndarray):
            self.data = data
            self.names = np.arange(data.shape[1]) if names is None else names

        if alpha < 0 or alpha > 1:
            raise ValueError(
                f"alpha must be larger than zero and less than 1, found {alpha}"
            )
        if not isinstance(B, int):
            try:
                B = int(B)
            except Exception as identifier:
                raise RuntimeError(
                    f"Bootstrap size B must be a integer, fail to convert", identifier
                )
        if B < 1:
            raise ValueError(f"Bootstrap size B must be larger than 1, found {B}")
        if not isinstance(w, int):
            try:
                w = int(w)
            except Exception as identifier:
                raise RuntimeError(
                    f"Bootstrap block size w must be a integer, fail to convert",
                    identifier,
                )
        if w < 1:
            raise ValueError(f"Bootstrap block size w must be larger than 1, found {w}")

        if algorithm not in ["R", "SQ"]:
            raise TypeError(f"Only R and SQ algorithm supported, found {algorithm}")

        self.alpha = alpha
        self.B = B
        self.w = w
        self.algorithm = algorithm

    def run(self):
        included, excluded, pvals = MCS(
            self.data, self.alpha, self.B, self.w, self.algorithm
        )

        self.included = self.names[included].ravel().tolist()
        self.excluded = self.names[excluded].ravel().tolist()
        self.pvalues = pd.Series(pvals.ravel(), index=self.excluded + self.included)
        return self

In [2]:
# We compute the MCS for all the ANN models and each crypto.
# We provide each crypto models losses and results in a subset of superior models
for each in crypto_list:
    crypto = each.split("/")[4]
    models = [filename for filename in os.listdir(f"{each}csv/") if filename.endswith( '.csv' )]
    df_main = read_csv_files_and_create_one_df(models, each)
    col_to_drop_containing_nas = df_main.columns[df_main.isna().any()].tolist()
    df_main.drop(col_to_drop_containing_nas, axis=1, inplace=True)
    df_main.to_csv(f'./mcs/losses/ann/{crypto}.csv')
    print(f"Processing {crypto}")
    print("shape",df_main.shape)
    print(f"model size : {len(df_main)}")
    mcs = ModelConfidenceSet(df_main, 0.1, 30, len(df_main.index)//20 , algorithm="SQ").run()
    print(f"MCS included {mcs.included}")
    print(f"MCS included {mcs.pvalues}")    
    df_results = pd.DataFrame({"rank": range(1,len(mcs.pvalues)+1),"mcs.pvalues":mcs.pvalues.values[::-1]}, index=mcs.pvalues.index[::-1])
    df_results.to_csv(f"./mcs/results/ann-{crypto}.csv")

NameError: name 'crypto_list' is not defined

# we compute all models results

In [29]:
def get_ann(x, arg):
    my_arg_list = x.split('-')
    if len(my_arg_list) > 4:
        return(my_arg_list[arg])
    else:
        return("-") 

In [30]:
all_results = [each for each in os.listdir('./mcs/results/') if each.startswith('ann')]

In [31]:
all_results[3]

'ann-BTC.csv'

In [32]:
data_frame = pd.read_csv(f'./mcs/results/{all_results[3]}')
data_frame

,Unnamed: 0,rank,mcs.pvalues
0,model5-drate-0.5-opt-rmsprop-garch-1-1-normal,1,1.000000
1,model5-drate-0.4-opt-rmsprop-garch-1-1-normal,2,0.066667
2,model3-drate-0.4-opt-sgd-garch-1-1-normal,3,0.066667
3,model5-drate-0.4-opt-rmsprop-egarch-1-1-skewst...,4,0.066667
4,model5-drate-0.5-opt-rmsprop-egarch-1-1-skewst...,5,0.000000
5,model3-drate-0.4-opt-sgd-egarch-1-1-skewstudent,6,0.000000
6,model3-drate-0.5-opt-sgd-egarch-1-1-skewstudent,7,0.000000
7,model3-drate-0.5-opt-sgd-garch-1-1-normal,8,0.000000
8,model4-drate-0.5-opt-sgd-garch-1-1-normal,9,0.000000
9,model4-drate-0.4-opt-sgd-garch-1-1-normal,10,0.000000


In [35]:
for each in all_results:
    crypto = each.split('-')[1].split('.')[0]
    data_frame = pd.read_csv(f'./results/{each}')
    data_frame['TS Model'] = data_frame.iloc[:,0].apply(lambda x: x.split('-')[len(x.split('-'))-4].capitalize())
    data_frame['p'] = data_frame.iloc[:,0].apply(lambda x: x.split('-')[len(x.split('-'))-3].capitalize())
    data_frame['q'] = data_frame.iloc[:,0].apply(lambda x: x.split('-')[len(x.split('-'))-2].capitalize())
    data_frame['Distribution'] = data_frame.iloc[:,0].apply(lambda x: x.split('-')[len(x.split('-'))-1].capitalize())
    data_frame['ANN Model'] = data_frame.iloc[:,0].apply(lambda x: f"{get_ann(x,0).capitalize()[:len(get_ann(x,0))-1]} {get_ann(x,0).capitalize()[len(get_ann(x,0))-1]}")
    data_frame['Optimizer'] = data_frame.iloc[:,0].apply(lambda x: get_ann(x,4))
    data_frame['Drop. Rate'] = data_frame.iloc[:,0].apply(lambda x: get_ann(x,2))
    data_frame['mcs.pvalues'] = data_frame['mcs.pvalues'].apply(lambda x: round(x,2))
    data_frame = data_frame[['rank','ANN Model','TS Model','Optimizer','Drop. Rate','p','q','Distribution','mcs.pvalues']]
    data_frame.columns = ['Rank','ANN Model','TS Model','Optimizer','Drop. Rate','p','q','Distribution','MCS p Values']
    data_frame.head(5).to_csv(f'./mcs/results/{crypto}-top-5-formatted.csv', index=False)